# Вычисление показателей для региона (на 3 уровнях) и для подаваемой территории

In [1]:
# Импорты

import sys
sys.path.append('../')
import transport_frames.src.graph_builder.criteria as criteria
import transport_frames.src.metrics.indicators as indicators
import transport_frames.src.metrics.grade_territory as grade_territory
import transport_frames.src.graph_builder.graphbuilder as graphbuilder
import momepy
import osmnx as ox
import geopandas as gpd
import shapely
import pandas as pd
import networkx as nx
import numpy as np
from shapely import wkt

/Users/sashamorozov/Documents/Code/NCCR/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# CRS (в формате ESPG) и OSM ID (id полигона региона, id точки админ. центра региона) по тестовым регионам

# Ленинградская_область EPSG: 32636  / 176095 / N27025179 (Гатчина)
# Санкт_Петербург EPSG:32636 / 337422 / N27490597
# Москва EPSG:32637 / 102269 / N1686293227
# Волгоградская_область EPSG:32638 / 77665 /  N27504363
# Тульская_область EPSG:32637 /  81993 / N34389350
# Омская_область EPSG:32643 / 140292 /  N27503946
# Краснодарский_край EPSG:32637 / 108082 / N27505129
# Тюменская_область EPSG:32642 / 140291 / N27505666
# Московская_область EPSG:32637 / 51490 / N1686293227

In [2]:
region_name = 'Ленинградская_область'
local_crs = 32636
geocode = 176095
region_capital = ox.geocode_to_gdf('N27025179',by_osmid=True)
city = ox.geocode_to_gdf(f'R{geocode}', by_osmid=True).to_crs(epsg=4326)
city['layer'] = region_name
city['status'] = 'region'

In [3]:
# Считывание входных данных

PLACEHOLDER = gpd.GeoDataFrame(geometry=[]) # если нет данных о сервисе

# полигоны районов и муниципальных образований
settlement = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_settlement.geojson')
district = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_district.geojson')

# населенные пункты и админ.центры точками
region_points = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_region_points.geojson')
admin_centers = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_district_admin_centres_18_nodes.geojson')
# либо:
# admin_centers = region_points[region_points['is_admin_centre_district']==True]


# Сервисы
railway_stations = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_railway_stations.geojson')
railway_roads = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_railway_roads.geojson')
bus_stops = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_bus_stops.geojson')
bus_routes = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_bus_routes.geojson')
fuel_stations = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_fuel_stations.geojson')
# ferry_terminal = PLACEHOLDER
ferry_terminal = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_ferry_terminal.geojson')
local_aerodrome = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_local_aerodrome.geojson')
international_aerodrome = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_international_aerodrome.geojson')
# либо:
# international_aerodrome = local_aerodrome[local_aerodrome['aerodrome:type']=='international']
# international_aerodrome['geometry'] = shapely.centroid(international_aerodrome['geometry']).set_crs(international_aerodrome.crs)

# Природные объекты
water = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_water.geojson')
nature_reserve = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/{region_name}/{region_name}_nature_reserve.geojson')

# Кастомная территория
custom_territory = gpd.read_file('/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/Ленинградская_область/project Светогорского поселения.geojson')
custom_territory_center = custom_territory.geometry.representative_point()
custom_territory_center = gpd.GeoDataFrame([{'geometry': custom_territory_center.iloc[0]}], crs=custom_territory_center.crs).to_crs(local_crs)

# Полигон РФ, необходим для построения транспортного каркаса
russia = ox.geocode_to_gdf("Russia") 
regions = gpd.read_file(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/data/regions_of_russia.geojson') #  get regions
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

In [4]:
# Загрузка УДС графа из файла 

citygraph = nx.read_graphml(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/graphs/{region_name}/{region_name}_uds.graphml')
citygraph = graphbuilder.convert_list_attr_from_str(citygraph)
citygraph = indicators.prepare_graph(citygraph)

In [5]:
# Загрузка интермодального графа из файла 

inter = nx.read_graphml(f'/Users/sashamorozov/Documents/НЦКР платформа ЦУ/Показатели по 9 регионам/data/graphs/{region_name}/{region_name}_inter.graphml')
inter = graphbuilder.convert_list_attr_from_str(inter)
inter = indicators.prepare_graph(inter)

In [5]:
citygraph_n,citygraph_e = momepy.nx_to_gdf(citygraph)

In [6]:
services = {
    'railway_stops' : railway_stations, # ЖД остановки
    'fuel' : fuel_stations, # АЗС станции
    'ports' : ferry_terminal, # Порты
    'local_aero' : local_aerodrome, # Аэропорты местного значения
    'international_aero' : international_aerodrome, # Аэропорты международного значения
    'oopt' : nature_reserve, # Особо охраняемые природные объекты
    'water_objects' : water, # Водные объекты
    'train_paths' : railway_roads, # ЖД пути
    'bus_stops' : bus_stops, # Автобусные остановки 
    'capital': region_capital, # Столица региона (точка)
    'points': region_points, # Населенные пункты региона (точки)
    'bus_routes' : bus_routes, # Автобусные маршруты
    'region_centers': admin_centers, # Административные центры муниципальных районов региона (точки)
    'neud_center': custom_territory_center # Центроида полигона кастомной территории (территория+3 км буфер)
}

In [7]:
G = indicators.assign_services_names_to_nodes(services,citygraph_n,citygraph,crs = local_crs)

railway_stops
fuel
ports
local_aero
international_aero
no oopt (((((((
no water_objects (((((((
no train_paths (((((((
bus_stops
capital
points
bus_routes
region_centers
neud_center


In [9]:
# Вычисление показателей на 3 уровнях: регион, районы, муниципальные образования

merged = indicators.indicator_area(G,inter,services,[city,district,settlement],local_crs)

100%|██████████| 2734/2734 [20:50<00:00,  2.19it/s]


new version
new version


/Users/sashamorozov/Documents/Code/NCCR/transport_frames/transport_frames/src/metrics/indicators.py:779: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged[cols_to_format] = merged[cols_to_format].applymap(
/Users/sashamorozov/Documents/Code/NCCR/transport_frames/transport_frames/src/metrics/indicators.py:779: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged[cols_to_format] = merged[cols_to_format].applymap(
/Users/sashamorozov/Documents/Code/NCCR/transport_frames/transport_frames/src/metrics/indicators.py:779: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged[cols_to_format] = merged[cols_to_format].applymap(


In [19]:
merged[2].to_parquet('Ленинградская_область_settlement.parquet', index=False)

In [8]:
# Вычисление показателей на уровне подаваемой территории+3км

temp = indicators.indicator_territory(G,custom_territory,local_crs,district,region_points,admin_centers,services,citygraph_n,citygraph_e)
temp

100%|██████████| 1/1 [00:00<00:00,  2.15it/s]
/Users/sashamorozov/Documents/Code/NCCR/transport_frames/transport_frames/src/metrics/indicators.py:871: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  filtered_region_centers = region_centers[region_centers.buffer(0.1).intersects(filtered_regions_terr.unary_union)]
/Users/sashamorozov/Documents/Code/NCCR/transport_frames/transport_frames/src/metrics/indicators.py:872: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  filtered_settlement_centers = points[points.buffer(0.1).intersects(filtered_regions_terr.unary_union)]


nearest_region_centers
nearest_settlement_centers


,name,to_region_admin_center,to_reg1,fuel_stations_accessibility,train_stops_accessibility,local_aero_accessibility,international_aero_accessibility,ports_accessibility,water_objects_accessibility,oopt_accessibility,...,number_of_international_aero,number_of_bus_stops,number_of_water_objects,number_of_oopt,density,train_path_length,number_of_bus_routes,to_nearest_settlement_center,to_nearest_mo_center,geometry
17950,None,223.144,5.102,7.097,45.651,107.438,172.108,197.244,0.032,0.268,...,0,0,1,0,1631.604,0.0,39,5065.79,57079.076,"POLYGON ((28.88155 61.13215, 28.91140 61.11113..."
